# Gossip Semantic Search

In [10]:
# %pip install feedparser
# %pip install -U sentence-transformers
# %pip install tf-keras

In [2]:
import feedparser
import pandas as pd
import json
import numpy as np

from datetime import datetime

## Dataset

## Load the Data

In [3]:
output_path_vsd = "../gossip_scrapper/vsd_articles.jl"

data = []
with open(output_path_vsd, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Erreur de décodage JSON : {e}")

df = pd.DataFrame(data)

df[:10]

,title,url,date,summary
0,N/A,N/A,N/A,N/A
1,N/A,N/A,N/A,N/A
2,N/A,N/A,N/A,N/A
3,N/A,N/A,N/A,N/A
4,N/A,N/A,N/A,N/A
5,N/A,N/A,N/A,N/A
6,"Décès d'un membre du ""Club Dorothée"", l'animat...",https://vsd.fr/70514-deces-dun-membre-du-club-...,2024-10-20T10:30:00+00:00,"Ce samedi 20 octobre, Dorothée est en deuil. A..."
7,N/A,N/A,N/A,N/A
8,"""Gosse de 12 ans"", ""aucune culture"" Quand Thie...",https://vsd.fr/70546-gosse-de-12-ans-aucune-cu...,2024-10-21T08:30:00+00:00,"Invité dans l'émission ""Cmédiatique"", Thierry ..."
9,N/A,N/A,N/A,N/A


### Add RSS feed

In [7]:
rss_feeds = [
    "https://www.vsd.fr/rss",
    "https://www.public.fr/rss"
]

articles = []

for feed_url in rss_feeds:
    feed = feedparser.parse(feed_url)
    for entry in feed.entries:
        articles.append({
            'title': entry.title,
            'link': entry.link,
            'published': entry.published,
            'summary': entry.summary
        })

df_rss = pd.DataFrame(articles)

print(f"Number of articles: {len(df_rss)}")
display(df_rss.head())


Number of articles: 100


,title,link,published,summary
0,"Tuerie dans le Nord : cinq personnes abattues,...",https://vsd.fr/73477-tuerie-dans-le-nord-cinq-...,"Sun, 15 Dec 2024 18:27:00 +0000",Un jeune homme de 22 ans s'est rendu à la gend...
1,Miss France 2025 : Pourquoi Angélique Angarni-...,https://vsd.fr/73471-miss-france-2025-pourquoi...,"Sun, 15 Dec 2024 16:54:56 +0000","En 2011, Angélique Angarni-Filopon terminait p..."
2,Rohan Dennis percute sa femme en voiture et la...,https://vsd.fr/73463-rohan-dennis-percute-sa-f...,"Sun, 15 Dec 2024 11:33:34 +0000",Son avocat aurait déclaré que l'ancien champio...
3,Une exposition tout en douceur au Musée des Ar...,https://vsd.fr/73442-une-exposition-tout-en-do...,"Sat, 14 Dec 2024 17:00:00 +0000","Jusqu'au 22 juin prochain, une rétrospective p..."
4,Y-a t'il des Françaises parmi les femmes les p...,https://vsd.fr/73436-y-a-til-des-francaises-pa...,"Sat, 14 Dec 2024 16:00:00 +0000",Le classement 2023 des 100 femmes les plus pui...


In [8]:
def convert_date(date_str: str) -> str:
    date_obj = datetime.strptime(date_str, '%a, %d %b %Y %H:%M:%S %z')
    return date_obj.strftime('%Y-%m-%dT%H:%M:%S%z')[:-4] + '00:00'

df_rss = df_rss.rename(columns={
    'link': 'url',
    'published': 'date'
})

df_rss['date'] = df_rss['date'].apply(convert_date)
df = pd.concat([df, df_rss], ignore_index=True)

print(f"Number of articles: {len(df)}")
display(df.head())

Number of articles: 23986


,title,url,date,summary
0,N/A,N/A,N/A,N/A
1,N/A,N/A,N/A,N/A
2,N/A,N/A,N/A,N/A
3,N/A,N/A,N/A,N/A
4,N/A,N/A,N/A,N/A


### Data Cleaning

In [ ]:
df_cleaned = df.replace("N/A", pd.NA).dropna()
df_cleaned = df_cleaned.drop_duplicates(subset=['url'])

print(f"Number of articles: {len(df_cleaned)}")
display(df_cleaned.head())

df_cleaned.to_csv('../data/articles.csv', index=False)

Number of articles: 17397


,title,url,date,summary
6,"Décès d'un membre du ""Club Dorothée"", l'animat...",https://vsd.fr/70514-deces-dun-membre-du-club-...,2024-10-20T10:30:00+00:00,"Ce samedi 20 octobre, Dorothée est en deuil. A..."
8,"""Gosse de 12 ans"", ""aucune culture"" Quand Thie...",https://vsd.fr/70546-gosse-de-12-ans-aucune-cu...,2024-10-21T08:30:00+00:00,"Invité dans l'émission ""Cmédiatique"", Thierry ..."
10,"""Un squelette"" Adeline Toniutti sans tabou sur...",https://vsd.fr/70600-un-squelette-adeline-toni...,2024-10-22T07:30:00+00:00,"Invitée chez Jordan de Luxe, Adeline Toniutti ..."
11,"""Sale"", ""Malaisant"" Cyril Hanouna écœuré, répo...",https://vsd.fr/70602-sale-malaisant-cyril-hano...,2024-10-22T09:30:00+00:00,Quelques heures après les propos chocs de Thie...
12,"""Bizarre"", ""pas fan"" Loana brise le silence su...",https://vsd.fr/70603-bizarre-pas-fan-loana-bri...,2024-10-22T10:30:00+00:00,"Dans les colonnes de ""Voici"", Loana se confie ..."


## Sentence-transformer model

In [66]:
from sentence_transformers import SentenceTransformer

sentences = df_cleaned['title'].tolist()
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)

np.save('../data/embeddings.npy', embeddings)

In [70]:
embeddings = np.load('../data/embeddings.npy')

print(embeddings.shape)

(17397, 384)


# Semantic Search

### Similarity Test

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

similarité = cosine_similarity([embeddings[0]], [embeddings[10]])
print(f"similarity between two embeddings is : {similarité[0][0]:.4f}")

Similarité entre la première et la dixième phrase : 0.4322


## Get Most Similar Function

In [123]:
from typing import List, Dict, Any

def get_most_similar(sentence:str,
                     embeddings: List[List[float]],
                     model: SentenceTransformer,
                     articles: pd.DataFrame,
                     n:int=5) -> List[Dict[str, Any]]:
    """
    Find the articles most similar to a given sentence using embeddings and cosine similarity.

    Args:
        sentence (str): The query sentence.
        embeddings (List[List[float]]): List of embeddings representing the articles.
        model (SentenceTransformer): The sentence transformer model.
        articles (pd.DataFrame): DataFrame containing the articles with columns ('title', 'url', 'date', 'summary').
        n (int, optional): Number of similar articles to return, defaults to 5.

    Returns:
        List[Dict[str, Any]]: List of dictionaries containing the most similar articles :
            - 'title' (str)
            - 'link' (str)
            - 'date' (str), formatted as 'dd/mm/yy'.
            - 'description
    """

    query_embedding = model.encode(sentence).reshape(1, -1)
    
    similarities = cosine_similarity(np.array(embeddings), query_embedding).flatten()
    
    top_n_indices = similarities.argsort()[-n:][::-1]
    
    most_similar = []
    for idx in top_n_indices:
        article = articles.iloc[idx]
        date = datetime.strptime(article['date'], '%Y-%m-%dT%H:%M:%S%z')
        formatted_date = date.strftime('%d/%m/%y')
        most_similar.append({
            'title': article['title'],
            'url': article['url'], 
            'date': formatted_date,
            'description': article['summary']
        })
    
    return most_similar

In [124]:
sentence = "michel sardou"
most_sim = get_most_similar(sentence, embeddings, model, df_cleaned)
for sim in most_sim:
    print(sim)

{'title': '"Bien fait pour sa gueule" : Michel Sardou s’en prend à Emmanuel Macron', 'url': 'https://www.public.fr/bien-fait-pour-sa-gueule-michel-sardou-sen-prend-a-emmanuel-macron', 'date': '13/12/24', 'description': 'Ce vendredi 13 décembre 2024, Nathalie Renoux a reçu Michel Sardou pour un entretien diffusé dans le "12.45" de M6. L’occasion pour le chanteur d’évoquer la politique d’Emmanuel Macron et la situation de la France aujourd’hui.'}
{'title': 'Michel Sardou : son ami d’enfance fait une confidence déroutante sur son plus gros tube', 'url': 'https://www.public.fr/michel-sardou-son-ami-denfance-fait-une-confidence-deroutante-sur-son-plus-gros-tube', 'date': '14/12/24', 'description': "À l'occasion de la diffusion de l'ultime concert de Michel Sardou, son ami d'enfance a fait des révélations étonnantes sur l'origine de son titre le plus populaire."}
{'title': "People La déclaration d'amour de Michel Sardou pour Brigitte Macron - VSD", 'url': 'https://vsd.fr/31183-people-la-decl